In [1]:
# %% imports & setup
import os, math, time, random
from dataclasses import dataclass
from pathlib import Path
from typing import List, Tuple, Iterable, Optional, Dict

import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, PeftModel
from opacus.accountants.rdp import RDPAccountant

import evaluate
rouge = evaluate.load("rouge")
bleu  = evaluate.load("sacrebleu")

import wandb

# --- W&B env (set your key via env var for security) ---
os.environ.setdefault("WANDB_ENTITY",  "bi000050-university-of-minnesota")
os.environ.setdefault("WANDB_PROJECT", "userdp_ft")
os.environ.setdefault("WANDB_MODE",    "online")  # or "offline"
wandb_api_key = os.environ.get("WANDB_API_KEY", "0d32ee09cbe7bae59ddac577f978a4f31cc4b559")
wandb.login(key=wandb_api_key)
USE_WANDB = True

# (optional) reduce fragmentation
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


wandb: Currently logged in as: bi000050 (bi000050-university-of-minnesota). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /users/2/bi000050/.netrc


Device: cuda


In [2]:
# %% config (OOM-safe defaults; you can scale up later)
@dataclass
class CFG:
    # choose scenario: "matched_epsilon_record" | "matched_epsilon_user" | "noise_stress"
    scenario: str = "matched_epsilon_record"

    # model / LoRA
    base_model: str = "gpt2"   # switch to "gpt2" after validating
    use_8bit: bool = False
    use_lora: bool = False            # LoRA drastically cuts memory
    lora_r: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.05
    lora_targets: Tuple[str, ...] = ("c_attn", "c_fc", "c_proj")

    # data
    dataset_name: str = "yahma/alpaca-cleaned"
    max_train_samples: int = 2000
    max_eval_samples: int = 500
    max_seq_len: int = 256          # reduce if memory tight (e.g., 192 or 128)

    # training
    epochs: int = 10
    micro_batch: int = 4            # small to allow per-example grads
    lr_sgd: float = 1e-4
    lr_disfom: float = 1e-4
    weight_decay: float = 0.0

    # DP privacy knobs
    clip_norm: float = 5
    noise_multiplier: float = 3   # used in noise_stress scenario
    delta: float = 1e-5
    target_epsilon: float = 1     # used in matched_epsilon_* scenarios

    # DISFOM
    rho_hat: float = 1
    box_lo: Optional[float] = None
    box_hi: Optional[float] = None

    # noise_stress sweep
    # stress_sigmas: Tuple[float, ...] = (0.6, 0.8, 1.0, 1.2)

    # IO
    save_root: str = "dp_llm_runs"
    log_csv: str = "dp_llm_compare.csv"

CFG = CFG()
Path(CFG.save_root).mkdir(exist_ok=True)
CFG


CFG(scenario='matched_epsilon_record', base_model='gpt2', use_8bit=False, use_lora=False, lora_r=16, lora_alpha=32, lora_dropout=0.05, lora_targets=('c_attn', 'c_fc', 'c_proj'), dataset_name='yahma/alpaca-cleaned', max_train_samples=2000, max_eval_samples=500, max_seq_len=256, epochs=10, micro_batch=4, lr_sgd=0.0001, lr_disfom=0.0001, weight_decay=0.0, clip_norm=5, noise_multiplier=3, delta=1e-05, target_epsilon=1, rho_hat=1, box_lo=None, box_hi=None, save_root='dp_llm_runs', log_csv='dp_llm_compare.csv')

In [3]:
# %% dataset, tokenization, optional synthetic user IDs (for user-level DP)
def format_alpaca(e):
    instr = e.get("instruction",""); inp = e.get("input",""); out = e.get("output","")
    if inp:
        prompt = f"### Instruction:\n{instr}\n\n### Input:\n{inp}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instr}\n\n### Response:\n"
    return {"text": prompt + out}

def build_dataset_and_loaders(add_user_ids: bool = False):
    raw = load_dataset(CFG.dataset_name)
    raw = raw.map(format_alpaca, remove_columns=raw["train"].column_names)

    def take_first(ds, n): return ds.select(range(min(n, len(ds))))
    train_raw = take_first(raw["train"], CFG.max_train_samples)
    eval_raw  = take_first(raw["train"].select(range(CFG.max_train_samples, len(raw["train"]))), CFG.max_eval_samples)
    dataset = DatasetDict({"train": train_raw, "validation": eval_raw})

    total_users = None
    if add_user_ids:
        def add_synth_user(ds, lam=5, seed=42):
            rng = np.random.default_rng(seed)
            n = len(ds); user_ids=[]; uid=0; i=0
            while i<n:
                k = max(1, int(rng.poisson(lam)))
                for _ in range(k):
                    if i>=n: break
                    user_ids.append(uid); i+=1
                uid+=1
            return ds.add_column("user_id", user_ids[:n]), uid
        dataset["train"], total_users = add_synth_user(dataset["train"], lam=5, seed=42)
        dataset["validation"], _ = add_synth_user(dataset["validation"], lam=5, seed=43)

    tok = AutoTokenizer.from_pretrained(CFG.base_model, use_fast=True)
    if tok.pad_token is None: tok.pad_token = tok.eos_token

    def tok_map(batch):
        x = tok(batch["text"], padding="max_length", truncation=True, max_length=CFG.max_seq_len)
        x["labels"] = x["input_ids"].copy()
        if "user_id" in batch:
            x["user_id"] = batch["user_id"]
        return x

    tokd = dataset.map(tok_map, batched=True)

    base_collator = DataCollatorForLanguageModeling(tok, mlm=False)

    class KeepUserID:
        def __init__(self, base): self.base = base
        def __call__(self, features):
            keep_uid = "user_id" in features[0]
            uids = [f["user_id"] for f in features] if keep_uid else None
            feats = [{k:v for k,v in f.items() if k not in ("text","user_id")} for f in features]
            batch = self.base(feats)
            if keep_uid:
                batch["user_id"] = torch.tensor(uids, dtype=torch.long)
            return batch

    collator = KeepUserID(base_collator)

    train_loader = DataLoader(tokd["train"], batch_size=CFG.micro_batch,
                              shuffle=True, drop_last=True, collate_fn=collator)
    eval_loader  = DataLoader(tokd["validation"], batch_size=CFG.micro_batch,
                              shuffle=False, drop_last=False, collate_fn=collator)
    return dataset, train_loader, eval_loader, tok, total_users

need_user = (CFG.scenario == "matched_epsilon_user")
dataset, train_loader, eval_loader, tokenizer, TOTAL_USERS = build_dataset_and_loaders(add_user_ids=need_user)
print("Train N:", len(train_loader.dataset), "Eval N:", len(eval_loader.dataset), "Users:", TOTAL_USERS)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Train N: 2000 Eval N: 500 Users: None


In [4]:
# %% model builder (LoRA on/off + optional gradient checkpointing)
def build_model():
    load_kwargs = {}
    if CFG.use_8bit:
        load_kwargs.update(dict(load_in_8bit=True, device_map="auto"))
    model = AutoModelForCausalLM.from_pretrained(CFG.base_model, **load_kwargs)
    if CFG.use_lora:
        lcfg = LoraConfig(r=CFG.lora_r, lora_alpha=CFG.lora_alpha, lora_dropout=CFG.lora_dropout,
                          target_modules=list(CFG.lora_targets), bias="none", task_type="CAUSAL_LM")
        model = get_peft_model(model, lcfg)
        print("[Model] LoRA enabled.")
    else:
        for p in model.parameters(): p.requires_grad = True
        print("[Model] LoRA disabled (full fine-tune).")

    # (optional) gradient checkpointing for larger models
    try:
        model.config.use_cache = False
        model.gradient_checkpointing_enable()
        print("[Model] Gradient checkpointing enabled.")
    except Exception as e:
        print("[Model] Gradient checkpointing not available:", e)

    model.to(device).train()
    return model

def params_that_train(model):
    return [p for p in model.parameters() if p.requires_grad]


In [5]:
# %% low-memory helpers: no GPU cat, CPU clip/avg/noise
def zero_like_params(params: Iterable[torch.Tensor], device="cpu"):
    return [torch.zeros_like(p, device=device, dtype=p.dtype) for p in params]

def l2norm_list(params_list: List[torch.Tensor]) -> float:
    s = 0.0
    for t in params_list:
        s += float(t.float().pow(2).sum().item())
    return s ** 0.5

def scale_inplace(lst: List[torch.Tensor], scale: float):
    for i in range(len(lst)):
        lst[i].mul_(scale)

def add_inplace(dst: List[torch.Tensor], src: List[torch.Tensor], alpha: float = 1.0):
    for i in range(len(dst)):
        dst[i].add_(src[i], alpha=alpha)

def clone_to_cpu(lst: List[torch.Tensor]) -> List[torch.Tensor]:
    return [t.detach().to("cpu", non_blocking=True).clone() for t in lst]


In [6]:
# %% DP gradient oracles (record-level & user-level) -- MEMORY SAFE
def dp_gradient_oracle_record(model, batch, clip_C: float, noise_sigma: float) -> List[torch.Tensor]:
    """
    Per-example grads via re-forward (no retain_graph), clip/avg/noise on CPU.
    """
    model_device = next(model.parameters()).device
    params = params_that_train(model)

    ids  = batch["input_ids"]
    labs = batch["labels"]
    amsk = batch["attention_mask"]
    B = ids.size(0)

    sum_grad_cpu = zero_like_params(params, device="cpu")

    for i in range(B):
        model.zero_grad(set_to_none=True)
        out = model(
            input_ids=ids[i:i+1].to(model_device, non_blocking=True),
            labels=labs[i:i+1].to(model_device, non_blocking=True),
            attention_mask=amsk[i:i+1].to(model_device, non_blocking=True),
        )
        out.loss.backward()
        g_i_gpu = [p.grad if p.grad is not None else torch.zeros_like(p) for p in params]
        g_i = clone_to_cpu(g_i_gpu)

        norm = l2norm_list(g_i)
        scale = min(1.0, clip_C / (norm + 1e-12))
        scale_inplace(g_i, scale)

        add_inplace(sum_grad_cpu, g_i, alpha=1.0 / B)
        model.zero_grad(set_to_none=True)

    std = noise_sigma * clip_C / B
    noisy_avg_cpu = [g + torch.randn_like(g, device="cpu") * std for g in sum_grad_cpu]
    private_grad = [g.to(model_device, non_blocking=True) for g in noisy_avg_cpu]
    return private_grad

def dp_gradient_oracle_user(model, batch, clip_C_user: float, noise_sigma: float):
    """
    User-level DP: sum by user on CPU, clip per-user, average users, add noise (CPU).
    Returns (private_grad_on_device, U_B).
    """
    model_device = next(model.parameters()).device
    params = params_that_train(model)

    ids  = batch["input_ids"]
    labs = batch["labels"]
    amsk = batch["attention_mask"]
    uids = batch["user_id"]
    B = ids.size(0)

    uniq = torch.unique(uids).tolist()
    user_sum = {int(u): zero_like_params(params, device="cpu") for u in uniq}

    for i in range(B):
        u = int(uids[i].item())
        model.zero_grad(set_to_none=True)
        out = model(
            input_ids=ids[i:i+1].to(model_device, non_blocking=True),
            labels=labs[i:i+1].to(model_device, non_blocking=True),
            attention_mask=amsk[i:i+1].to(model_device, non_blocking=True),
        )
        out.loss.backward()
        g_i_gpu = [p.grad if p.grad is not None else torch.zeros_like(p) for p in params]
        g_i = clone_to_cpu(g_i_gpu)
        add_inplace(user_sum[u], g_i, alpha=1.0)
        model.zero_grad(set_to_none=True)

    clipped_users = []
    for u in uniq:
        g_u = user_sum[u]
        nrm = l2norm_list(g_u)
        scale = min(1.0, clip_C_user / (nrm + 1e-12))
        scale_inplace(g_u, scale)
        clipped_users.append(g_u)

    U_B = len(uniq)
    avg_cpu = zero_like_params(params, device="cpu")
    for g_u in clipped_users:
        add_inplace(avg_cpu, g_u, alpha=1.0 / U_B)

    std = noise_sigma * clip_C_user / U_B
    noisy_avg_cpu = [g + torch.randn_like(g, device="cpu") * std for g in avg_cpu]
    private_grad = [g.to(model_device, non_blocking=True) for g in noisy_avg_cpu]
    return private_grad, U_B


In [7]:
# %% updates: weight decay, DP-SGD, DP-DISFOM (prox on displacement, CPU-safe)
def apply_weight_decay(params, wd):
    if wd == 0: return
    with torch.no_grad():
        for p in params: p.add_(-wd * p)

def step_dpsgd(model, lr, private_grad):
    with torch.no_grad():
        for p, g in zip(params_that_train(model), private_grad):
            p.add_(-lr * g)

# DISFOM prox φ(z)=(ρ̂/2)||z||_1^2 applied to displacement u = -lr·ĝ
def tau_by_bisection(u: torch.Tensor, rho_hat: float, max_iter: int = 80, tol: float = 1e-12) -> float:
    lo, hi = 0.0, float(u.abs().max().item())
    def rhs(tau: float) -> float:
        return float(torch.nn.functional.relu(u.abs() - tau).sum().item())
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        R = mid - rho_hat * rhs(mid)
        if abs(R) <= tol: return mid
        if R > 0: hi = mid
        else:     lo = mid
    return 0.5*(lo+hi)

def step_dpdisfom(model, lr, private_grad, rho_hat, box_lo=None, box_hi=None):
    """
    CPU-friendly prox: flatten on CPU to avoid peak GPU memory.
    """
    params = params_that_train(model)
    dev = next(model.parameters()).device
    with torch.no_grad():
        # move to CPU
        xk_list = [p.data.detach().cpu().clone() for p in params]
        g_list  = [g.detach().cpu().clone() for g in private_grad]

        xk_vec = torch.cat([t.view(-1) for t in xk_list])
        g_vec  = torch.cat([t.view(-1) for t in g_list])
        u = (-lr) * g_vec

        if box_lo is None and box_hi is None:
            tau = tau_by_bisection(u, rho_hat)
            y = torch.sign(u) * torch.nn.functional.relu(u.abs() - tau)
            x_next = xk_vec + y
        else:
            l = torch.full_like(xk_vec, box_lo) if box_lo is not None else torch.full_like(xk_vec, -float("inf"))
            ubox = torch.full_like(xk_vec, box_hi) if box_hi is not None else torch.full_like(xk_vec,  float("inf"))
            l_disp, u_disp = l - xk_vec, ubox - xk_vec
            def rhs_box(tau: float) -> float:
                z = torch.sign(u) * torch.nn.functional.relu(u.abs() - tau)
                z = torch.max(torch.min(z, u_disp), l_disp)
                return float(z.abs().sum().item())
            lo, hi = 0.0, float(u.abs().max().item())
            for _ in range(80):
                mid = 0.5*(lo+hi)
                R = mid - rho_hat * rhs_box(mid)
                if abs(R) <= 1e-12: tau = mid; break
                if R > 0: hi = mid
                else:     lo = mid
            else:
                tau = 0.5*(lo+hi)
            z = torch.sign(u) * torch.nn.functional.relu(u.abs() - tau)
            y = torch.max(torch.min(z, u_disp), l_disp)
            x_next = xk_vec + y

        # write back to device params
        offset = 0
        for p in params:
            n = p.numel()
            chunk = x_next[offset:offset+n].view(p.shape).to(dev)
            p.copy_(chunk)
            offset += n


In [8]:
# %% eval: PPL + ROUGE/BLEU
def eval_perplexity(model, eval_loader):
    model.eval(); total, count = 0.0, 0
    with torch.no_grad():
        for batch in eval_loader:
            for k in batch: batch[k] = batch[k].to(device)
            out = model(**batch)
            total += float(out.loss.item()) * batch["input_ids"].size(0)
            count += batch["input_ids"].size(0)
    mean = total / max(1, count)
    ppl = math.exp(mean) if mean < 50 else float("inf")
    model.train()
    return mean, ppl

def split_prompt_and_ref(txt: str):
    key = "### Response:\n"
    if key in txt:
        i = txt.index(key)
        return txt[:i+len(key)], txt[i+len(key):].strip()
    return txt, ""

def collect_eval_prompts_and_refs(dataset_validation, max_items=300):
    prompts, refs = [], []
    m = min(len(dataset_validation), max_items)
    for i in range(m):
        txt = dataset_validation[i]["text"]
        p, r = split_prompt_and_ref(txt)
        prompts.append(p); refs.append(r)
    return prompts, refs

@torch.no_grad()
def generate_responses(model, tok, prompts, max_new_tokens=128, temperature=0.0, top_p=1.0):
    outs=[]
    for p in prompts:
        inputs = tok(p, return_tensors="pt").to(device)
        ids = model.generate(**inputs, max_new_tokens=max_new_tokens,
                             do_sample=(temperature>0), temperature=temperature, top_p=top_p,
                             pad_token_id=tok.eos_token_id)
        full = tok.decode(ids[0], skip_special_tokens=True)
        outs.append(full[len(p):].strip() if full.startswith(p) else full.strip())
    return outs

def eval_text_metrics(preds, refs):
    r = rouge.compute(predictions=preds, references=refs)
    b = bleu.compute(predictions=preds, references=[[x] for x in refs])
    return {**r, "bleu": b["score"]}


In [9]:
# %% privacy helper: σ for target ε (record-style search)
def sigma_for_epsilon_record(target_eps, delta, q, T, lo=0.3, hi=5.0):
    for _ in range(25):
        mid = 0.5*(lo+hi)
        acct = RDPAccountant()
        for _ in range(T): acct.step(noise_multiplier=mid, sample_rate=q)
        eps = acct.get_epsilon(delta=delta)
        if eps > target_eps: lo = mid
        else:                hi = mid
    return hi


In [10]:
# %% training loop (scenario-aware)
def train_once(method: str, sigma: float, train_loader, eval_loader,
               privacy_mode: str, total_users: Optional[int] = None) -> Dict[str, float]:
    assert privacy_mode in ("record","user")
    model = build_model()
    params = params_that_train(model)
    lr = CFG.lr_disfom if method == "dpdisfom" else CFG.lr_sgd

    acct = RDPAccountant()

    run = None
    if USE_WANDB:
        run = wandb.init(
            project=os.environ["WANDB_PROJECT"],
            entity=os.environ.get("WANDB_ENTITY"),
            name=f"{method}-{privacy_mode}-sigma{sigma}-{int(time.time())}",
            group=f"{CFG.scenario}",
            config={
                "base_model": CFG.base_model, "method": method,
                "epochs": CFG.epochs, "clip": CFG.clip_norm,
                "sigma": sigma, "delta": CFG.delta,
                "rho_hat": CFG.rho_hat if method=="dpdisfom" else None,
                "lr": lr, "batch": CFG.micro_batch, "seq_len": CFG.max_seq_len,
                "privacy_mode": privacy_mode, "use_lora": CFG.use_lora
            },
            reinit=True,
        )
        wandb.define_metric("train/iter")
        wandb.define_metric("train/loss_iter", step_metric="train/iter")
        wandb.define_metric("epoch")
        wandb.define_metric("metrics/epsilon", step_metric="epoch")
        wandb.define_metric("metrics/loss_epoch", step_metric="epoch")

    global_step = 0
    for ep in range(CFG.epochs):
        pbar = tqdm(train_loader, desc=f"{method} [{privacy_mode}] σ={sigma} | epoch {ep+1}/{CFG.epochs}")
        for batch in pbar:
            for k in batch: batch[k] = batch[k].to(device)

            if privacy_mode == "record":
                g_priv = dp_gradient_oracle_record(model, batch, CFG.clip_norm, sigma)
                sample_rate = CFG.micro_batch / len(train_loader.dataset)
            else:
                g_priv, U_B = dp_gradient_oracle_user(model, batch, CFG.clip_norm, sigma)
                if total_users is None or total_users == 0:
                    raise ValueError("total_users must be provided for user-level DP.")
                sample_rate = U_B / total_users

            # pre-update loss for logging (approx same step)
            with torch.no_grad():
                out = model(input_ids=batch["input_ids"], labels=batch["labels"], attention_mask=batch["attention_mask"])
                batch_loss = float(out.loss.detach().cpu().item())

            apply_weight_decay(params, CFG.weight_decay)
            if method == "dpsgd":
                step_dpsgd(model, lr, g_priv)
            elif method == "dpdisfom":
                step_dpdisfom(model, lr, g_priv, CFG.rho_hat, CFG.box_lo, CFG.box_hi)
            else:
                raise ValueError(method)

            acct.step(noise_multiplier=sigma, sample_rate=sample_rate)

            global_step += 1
            pbar.set_postfix(loss=f"{batch_loss:.4f}")
            if run is not None:
                wandb.log({"train/iter": global_step, "train/loss_iter": batch_loss}, step=global_step)

        eval_loss, eval_ppl = eval_perplexity(model, eval_loader)
        eps_now = acct.get_epsilon(delta=CFG.delta)
        if run is not None:
            wandb.log({"epoch": ep+1,
                       "metrics/loss_epoch": eval_loss,
                       "metrics/ppl": eval_ppl,
                       "metrics/epsilon": eps_now}, step=global_step)

    # save
    tag = f"{CFG.base_model.replace('/','_')}_{method}_{privacy_mode}_sigma{sigma}"
    outdir = Path(CFG.save_root)/tag
    outdir.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(outdir.as_posix())

    # final eval
    prompts, refs = collect_eval_prompts_and_refs(dataset["validation"], max_items=300)
    model.eval()
    preds = generate_responses(model, tokenizer, prompts, max_new_tokens=128, temperature=0.0)
    text_metrics = eval_text_metrics(preds, refs)
    ppl_loss, ppl = eval_perplexity(model, eval_loader)

    if run is not None:
        wandb.log({
            "final/rouge1": text_metrics["rouge1"],
            "final/rouge2": text_metrics["rouge2"],
            "final/rougeL": text_metrics["rougeL"],
            "final/bleu":   text_metrics["bleu"],
            "final/ppl_loss": ppl_loss,
            "final/ppl": ppl
        })
        run.finish()

    return {"model_dir": outdir.as_posix(),
            "rougeL": text_metrics["rougeL"],
            "bleu": text_metrics["bleu"],
            "ppl": ppl, "ppl_loss": ppl_loss}


In [11]:
# %% orchestrators
def run_matched_epsilon_record():
    print("== Matched-ε (Record-level DP) ==")
    steps_per_epoch = len(train_loader)
    T = CFG.epochs * steps_per_epoch
    q = CFG.micro_batch / len(train_loader.dataset)
    sigma = sigma_for_epsilon_record(CFG.target_epsilon, CFG.delta, q, T, lo=0.3, hi=5.0)
    print(f"ε_target={CFG.target_epsilon} -> σ≈{sigma:.3f} (q={q:.6f}, T={T})")

    res = {}
    for method in ("dpsgd", "dpdisfom"):
    # for method in ("dpdisfom",):
        res[method] = train_once(method, sigma, train_loader, eval_loader, privacy_mode="record")
        print(method, res[method])
    return res, sigma

def run_matched_epsilon_user():
    assert TOTAL_USERS is not None, "Rebuild dataset with add_user_ids=True."
    print("== Matched-ε (User-level DP) ==")
    steps_per_epoch = len(train_loader)
    T = CFG.epochs * steps_per_epoch
    # expected distinct users per batch ~ micro_batch (Poisson(5) gives many-user batches); conservative:
    q_user = min(1.0, CFG.micro_batch / max(1, TOTAL_USERS))
    sigma = sigma_for_epsilon_record(CFG.target_epsilon, CFG.delta, q_user, T, lo=0.3, hi=5.0)
    print(f"ε_target={CFG.target_epsilon} -> σ≈{sigma:.3f} (q_user≈{q_user:.6f}, T={T}, users={TOTAL_USERS})")

    res = {}
    for method in ("dpsgd", "dpdisfom"):
    # for method in ("dpdisfom",):
        res[method] = train_once(method, sigma, train_loader, eval_loader, privacy_mode="user", total_users=TOTAL_USERS)
        print(method, res[method])
    return res, sigma

def run_noise_stress():
    print("== Noise-Stress (σ sweep) ==")
    table = []
    for sigma in CFG.stress_sigmas:
        for method in ("dpsgd", "dpdisfom"):
            res = train_once(method, sigma, train_loader, eval_loader, privacy_mode="record")
            print(f"sigma={sigma:.2f}", method, res)
            table.append({"sigma": sigma, "method": method, **res})
    return table


In [12]:
# %% execute chosen scenario
if CFG.scenario == "matched_epsilon_record":
    results, sigma_star = run_matched_epsilon_record()
elif CFG.scenario == "matched_epsilon_user":
    results, sigma_star = run_matched_epsilon_user()
elif CFG.scenario == "noise_stress":
    stress_table = run_noise_stress()
else:
    raise ValueError("Unknown CFG.scenario")


== Matched-ε (Record-level DP) ==


/users/2/bi000050/.conda/envs/opt/lib/python3.12/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


ε_target=1 -> σ≈1.005 (q=0.002000, T=5000)
[Model] LoRA disabled (full fine-tune).
[Model] Gradient checkpointing enabled.


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


dpsgd [record] σ=1.00489399433136 | epoch 1/10:   0%|          | 0/500 [00:00<?, ?it/s]

...

dpdisfom [record] σ=1.00489399433136 | epoch 9/10:   0%|          | 0/500 [00:00<?, ?it/s]

dpdisfom [record] σ=1.00489399433136 | epoch 10/10:   0%|          | 0/500 [00:00<?, ?it/s]

/users/2/bi000050/.conda/envs/opt/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


epoch,▁▂▃▃▄▅▆▆▇█
final/bleu,▁
final/ppl,▁
final/ppl_loss,▁
final/rouge1,▁
final/rouge2,▁
final/rougeL,▁
metrics/epsilon,▁▂▃▃▄▅▆▆▇█
metrics/loss_epoch,██▁▁▃▂▃▂▄▄
metrics/ppl,██▁▁▃▂▃▂▄▄
train/iter,▁▁▁▁▁▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇████


dpdisfom {'model_dir': 'dp_llm_runs/gpt2_dpdisfom_record_sigma1.00489399433136', 'rougeL': np.float64(0.12679352260761162), 'bleu': 2.7675663702625712, 'ppl': 20.11257684560192, 'ppl_loss': 3.001345333099365}
